<a href="https://colab.research.google.com/github/moshekranc-git/pythonplayground/blob/master/SpeechRecognition1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Install all needed python libraries

In [1]:
!pip install SpeechRecognition
!pip install pydub
!pip install moviepy
#!pip install wheel
#!pip install pocketsphinx-0.1.15-cp27-cp27m-macosx_10_10_x86_64.whl


     |████████████████████████████████| 32.8MB 125kB/s 


#2. Upload the video file to be analyzed into the colab environment
To download a file from YouTube to your local mac machine, use ClipGrab.
See https://www.wikihow.com/Download-YouTube-Videos-on-a-Mac

To upload the video file from your local machine to colab:
Click on the folder icon on the left, then use the interface to upload the file. (Remember: you will need to upload the file each time you reinstantiate colab.)

For this notebook, I will be using "Alphabet 2019 Q3 Earnings Call.mp4" which can be found at https://www.youtube.com/watch?v=1vj7AjthgjE


#3. Extract the sound from the video file in a format that the speech recognition library can understand.
Most speech recognition libraries require a lossless codec to transmit audio. FLAC or LINEAR16 is recommended.

In [19]:
!ffmpeg -i "Alphabet 2019 Q3 Earnings Call.mp4" "Alphabet 2019 Q3 Earnings Call.flac"

#  Just to be sure, download to my machine and listen to make sure it worked
#from google.colab import files
#files.download("Alphabet 2019 Q3 Earnings Call.flac")

ffmpeg version 3.4.6-0ubuntu0.18.04.1 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.3.0-16ubuntu3)
  configuration: --prefix=/usr --extra-version=0ubuntu0.18.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --ena

----------------------------------------
Exception happened during processing of request from ('::ffff:127.0.0.1', 36234, 0, 0)
Traceback (most recent call last):
  File "/usr/lib/python3.6/socketserver.py", line 320, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 351, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 364, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.6/socketserver.py", line 724, in __init__
    self.handle()
  File "/usr/lib/python3.6/http/server.py", line 418, in handle
    self.handle_one_request()
  File "/usr/lib/python3.6/http/server.py", line 406, in handle_one_request
    method()
  File "/usr/lib/python3.6/http/server.py", line 639, in do_GET
    self.copyfile(f, self.wfile)
  File "/usr/lib/python3.6/http/server.py", line 800, in copyfile
    shutil.copyfil

#4. Convert speech to text
See https://stackabuse.com/introduction-to-speech-recognition-with-python/ , 
https://becominghuman.ai/the-ultimate-guide-to-speech-recognition-with-python-d45256c73e54 ,
https://github.com/Uberi/speech_recognition/blob/master/reference/library-reference.rst, https://springml.com/blog/text-classification-natural-language-api/


This requires a paying account on Google - the free versions will not handle long audio. After creating an account and creating credentials, upload the json credentials file to colab, then pass the file name as a parameter to recognize_google_cloud


In [22]:
# for test purposes, start with a small test file until you are sure everything works
!gsutil cp gs://cloud-samples-tests/speech/brooklyn.flac brooklyn.flac 
  

Copying gs://cloud-samples-tests/speech/brooklyn.flac...
/ [1 files][ 32.7 KiB/ 32.7 KiB]                                                
Operation completed over 1 objects/32.7 KiB.                                     


In [26]:
import requests
import io
import speech_recognition as sr
import json

# This function translate speech to text
def speech_to_text(file):
  recognizer = sr.Recognizer()
  audio = sr.AudioFile(file)
  with audio as source:
    speech = recognizer.record(source)
    try:
      with open("moshe1-aaae31fcbe39.json") as f: # your google API key here
        GOOGLE_CLOUD_SPEECH_CREDENTIALS = f.read()

      # Call recognizer with audio and google cloud credentials
      text = recognizer.recognize_google_cloud (speech, credentials_json=GOOGLE_CLOUD_SPEECH_CREDENTIALS)
      return text
    # If recognizer doesn't understand
    except sr.UnknownValueError: 
      print("Speech Recognition service could not understand audio") 
    except sr.RequestError as e: 
      print("Could not request results from Speech Recognition service; {0}".format(e)) 
      

how old is the Brooklyn Bridge 


###4a. Start with the small test file


In [0]:
text = speech_to_text("brooklyn.flac")
print(text)

##4b. Then graduate to the real file

In [0]:
#text = speech_to_text("Alphabet 2019 Q3 Earnings Call.flac")
print(text)

#5. Save text to a file for subsequent processing

In [0]:
#save the returned text to a file
text_file = open("brooklyn.txt", "w")
n = text_file.write(text)
text_file.close()


In [13]:
'''import sys
from moviepy.editor import *
video = VideoFileClip("Alphabet 2019 Q3 Earnings Call.mp4") # 2.
audio = video.audio # 3.
audio.write_audiofile("Alphabet1.pcm", codec="libx265") # 4. '''

import requests
import io
from pydub import AudioSegment

def mp4_to_wav_mem(file):
        audio = AudioSegment.from_file_using_temporary_files(file, 'mp4')
        file = io.BytesIO()
        file = audio.export(file, format="wav")
        file.seek(0)
        return file

def mp4_to_wav(file):
        audio = AudioSegment.from_file(file, format="mp4")
        audio.export("audio.wav", format="wav")
        return audio

f = open("Alphabet 2019 Q3 Earnings Call.mp4", "rb")
#file = mp4_to_wav_mem(f)
audio = mp4_to_wav(f)
audio.preview

CouldntDecodeError: ignored

In [17]:
!ffmpeg -i "Alphabet 2019 Q3 Earnings Call.mp4" "Alphabet 2019 Q3 Earnings Call.flac"

ffmpeg version 3.4.6-0ubuntu0.18.04.1 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.3.0-16ubuntu3)
  configuration: --prefix=/usr --extra-version=0ubuntu0.18.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --ena

In [0]:
import speech_recognition as sr 
r = sr.Recognizer()

In [3]:
import sys
from moviepy.editor import *

video = VideoFileClip("Alphabet 2019 Q3 Earnings Call.mp4") # 2.
audio = video.audio # 3.
audio.write_audiofile("Alphabet1.pcm", codec="libx265") # 4.

[MoviePy] Writing audio in Alphabet1.pcm


  0%|          | 0/72135 [00:00<?, ?it/s]


OSError: ignored

In [0]:
    import requests
    import io
    import speech_recognition as sr
    from pydub import AudioSegment


    # This function translate speech to text
    def speech_to_text(file):
        recognizer = sr.Recognizer()
        audio = sr.AudioFile(file)
        with audio as source:
            speech = recognizer.record(source)
            speech
            try:
                # Call recognizer with audio and language
                text = recognizer.recognize_google(speech, show_all=True)
                print("understood: " + text)
                return text
            # If recognizer doesn't understand
            except sr.UnknownValueError: 
              print("Google Speech Recognition could not understand audio") 
            except sr.RequestError as e: 
              print("Could not request results from Google Speech Recognition service; {0}".format(e)) 

    def mp4_to_wav(file):
        audio = AudioSegment.from_file(file, format="mp4")
        audio.export("audio.wav", format="wav")
        return audio

    def mp4_to_wav_mem(file):
        audio = AudioSegment.from_file_using_temporary_files(file, 'mp4')
        file = io.BytesIO()
        file = audio.export(file, format="wav")
        file.seek(0)
        return file

#url = 'https://www.youtube.com/channel/UC9RZRAPdKRM91k1XwbELJEg'
#r = requests.get(url, stream=True)


In [0]:

.content


b'\n    <!DOCTYPE html><html lang="en" data-cast-api-enabled="true"><head><style name="www-roboto" >@font-face{font-family:\'Roboto\';font-style:italic;font-weight:400;src:local(\'Roboto Italic\'),local(\'Roboto-Italic\'),url(//fonts.gstatic.com/s/roboto/v18/KFOkCnqEu92Fr1Mu51xIIzc.ttf)format(\'truetype\');}@font-face{font-family:\'Roboto\';font-style:italic;font-weight:500;src:local(\'Roboto Medium Italic\'),local(\'Roboto-MediumItalic\'),url(//fonts.gstatic.com/s/roboto/v18/KFOjCnqEu92Fr1Mu51S7ACc6CsE.ttf)format(\'truetype\');}@font-face{font-family:\'Roboto\';font-style:normal;font-weight:400;src:local(\'Roboto Regular\'),local(\'Roboto-Regular\'),url(//fonts.gstatic.com/s/roboto/v18/KFOmCnqEu92Fr1Mu4mxP.ttf)format(\'truetype\');}@font-face{font-family:\'Roboto\';font-style:normal;font-weight:500;src:local(\'Roboto Medium\'),local(\'Roboto-Medium\'),url(//fonts.gstatic.com/s/roboto/v18/KFOlCnqEu92Fr1MmEU9fBBc9.ttf)format(\'truetype\');}</style><script name="www-roboto" >if (document

In [0]:
f = open("Alphabet 2019 Q3 Earnings Call.mp4", "rb")
file = mp4_to_wav_mem(f)
speech_to_text(file)

Could not request results from Google Speech Recognition service; recognition connection failed: [Errno 32] Broken pipe


In [0]:
from google.cloud import speech_v1
from google.cloud.speech_v1 import enums


def sample_long_running_recognize(storage_uri):
    """
    Transcribe long audio file from Cloud Storage using asynchronous speech
    recognition

    Args:
      storage_uri URI for audio file in Cloud Storage, e.g. gs://[BUCKET]/[FILE]
    """

    client = speech_v1.SpeechClient()

    # storage_uri = 'gs://cloud-samples-data/speech/brooklyn_bridge.raw'

    # Sample rate in Hertz of the audio data sent
    sample_rate_hertz = 16000

    # The language of the supplied audio
    language_code = "en-US"

    # Encoding of audio data sent. This sample sets this explicitly.
    # This field is optional for FLAC and WAV audio formats.
    encoding = enums.RecognitionConfig.AudioEncoding.LINEAR16
    config = {
        "sample_rate_hertz": sample_rate_hertz,
        "language_code": language_code,
        "encoding": encoding,
    }
    audio = {"uri": storage_uri}

    operation = client.long_running_recognize(config, audio)

    print(u"Waiting for operation to complete...")
    response = operation.result()

    for result in response.results:
        # First alternative is the most probable result
        alternative = result.alternatives[0]
        print(u"Transcript: {}".format(alternative.transcript))
